In [5]:
# -*- coding: utf-8 -*-
"""Alexa_Reviews.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1GOEZ_dxxKdkA-Exe7GDP4k3oE6gLU5eO
"""

from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import seaborn as sns

data = pd.read_csv('drive/My Drive/Projects/Amazon Alexa Reviews/amazon_alexa.tsv', delimiter = '\t', quoting = 3)

data.shape

data.info()

data.head()

data.tail()

data.isnull().any()

data.describe()

data['length'] = data['verified_reviews'].apply(len)

data.shape

data.head()

data.groupby('rating').describe()

data.groupby('feedback').describe()

data['rating'].value_counts().plot.bar(color = 'magenta')
plt.title('Visualizing the ratings dist.')
plt.xlabel('ratings')
plt.ylabel('count')
plt.show()

data['rating'].value_counts()

labels = '5', '4', '3', '2', '1'
sizes = [2286, 455, 161, 152, 96]
colors = ['cyan', 'magenta', 'pink', 'yellow', 'red']
explode = [0.001, 0.001, 0.001, 0.001, 0.001]

plt.pie(sizes, labels = labels, colors = colors, explode = explode, shadow = True)
plt.title(' pie chart representing ratings occuposition')
plt.show()

data['variation'].value_counts().plot.bar(color = 'cyan', figsize = (11, 7))
plt.title('Visualizing the variations dist.')
plt.xlabel('variations')
plt.ylabel('count')
plt.show()

data['feedback'].value_counts().plot.bar(color = 'orange', figsize = (5, 4))
plt.title('Visualizing the feedbacks dist.')
plt.xlabel('feedbacks')
plt.ylabel('count')
plt.show()

data['length'].value_counts().plot.hist(color = 'pink', figsize = (12, 5), bins = 50)
plt.title('Visualizing the length dist.')
plt.xlabel('lengths')
plt.ylabel('count')
plt.show()

data.length.describe()

data[data['length'] == 1]['verified_reviews'].iloc[0]

data[data['length'] == 21]['verified_reviews'].iloc[0]

data[data['length'] == 50]['verified_reviews'].iloc[0]

data[data['length'] == 2853]['verified_reviews'].iloc[0]

data.date.describe()

data['date'].value_counts()

data.groupby('variation').mean()[['rating']].plot.bar(color = 'brown', figsize=(11, 6))
plt.title("Variation wise Mean Ratings")
plt.xlabel('variatiions')
plt.ylabel('ratings')
plt.show()

data.groupby('feedback').mean()[['rating']].plot.bar(color = 'crimson', figsize=(5, 4))
plt.title("feedback wise Mean Ratings")
plt.xlabel('feedbacks')
plt.ylabel('ratings')
plt.show()

data.groupby('length').mean()[['rating']].plot.hist(color = 'gray', figsize=(7, 6), bins = 20)
plt.title("length wise Mean Ratings")
plt.xlabel('length')
plt.ylabel('ratings')
plt.show()

from sklearn.feature_extraction.text import CountVectorizer


cv = CountVectorizer(stop_words = 'english')
words = cv.fit_transform(data.verified_reviews)
sum_words = words.sum(axis=0)


words_freq = [(word, sum_words[0, idx]) for word, idx in cv.vocabulary_.items()]
words_freq = sorted(words_freq, key = lambda x: x[1], reverse = True)
frequency = pd.DataFrame(words_freq, columns=['word', 'freq'])


frequency.head(20).plot(x='word', y='freq', kind='bar', figsize=(15, 7), color = 'lightgreen')
plt.title("Most Frequently Occuring Words - Top 20")

from wordcloud import WordCloud

wordcloud = WordCloud(background_color = 'lightcyan', width = 1200, height = 700).generate_from_frequencies(dict(words_freq))

plt.figure(figsize=(10,8))
plt.imshow(wordcloud)
plt.title("WordCloud - Vocabulary from Reviews", fontsize = 22)

# cleaning the texts
# importing the libraries for Natural Language Processing

import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

corpus = []

for i in range(0, 3150):
  review = re.sub('[^a-zA-Z]', ' ', data['verified_reviews'][i])
  review = review.lower()
  review = review.split()
  ps = PorterStemmer()
  review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
  review = ' '.join(review)
  corpus.append(review)

# creating bag of words

from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features = 2500)

x = cv.fit_transform(corpus).toarray()

y = data.iloc[:, 4].values

print(x.shape)
print(y.shape)

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 15)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

from sklearn.preprocessing import MinMaxScaler

mm = MinMaxScaler()

x_train = mm.fit_transform(x_train)
x_test = mm.transform(x_test)

"""**Random Forest**"""

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

model = RandomForestClassifier()
model.fit(x_train, y_train)

y_pred = model.predict(x_test)

print("Training Accuracy :", model.score(x_train, y_train))
print("Testing Accuracy :", model.score(x_test, y_test))

cm = confusion_matrix(y_test, y_pred)
print(cm)

# applying k fold cross validation

from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = model, X = x_train, y = y_train, cv = 10)

print("Accuracy :", accuracies.mean())
print("Standard Variance :", accuracies.std())

params = {
    'bootstrap': [True],
    'max_depth': [80, 100],
    'min_samples_split': [8, 12],
    'n_estimators': [100, 300]
}

# applying grid search with stratified folds

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

cv_object = StratifiedKFold(n_splits = 2)

grid = GridSearchCV(estimator = model, param_grid = params, cv = cv_object, verbose = 0, return_train_score = True)
grid.fit(x_train, y_train.ravel())

print("Best Parameter Combination : {}".format(grid.best_params_))

print("Mean Cross Validation Accuracy - Train Set : {}".format(grid.cv_results_['mean_train_score'].mean()*100))
print("Mean Cross Validation Accuracy - Validation Set : {}".format(grid.cv_results_['mean_test_score'].mean()*100))

from sklearn.metrics import accuracy_score
print("Accuracy Score for Test Set :", accuracy_score(y_test, y_pred))

"""**Xg Boost Model**"""

from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix

model = XGBClassifier()
model.fit(x_train, y_train)

y_pred = model.predict(x_test)

print("Training Accuracy :", model.score(x_train, y_train))
print("Testing Accuracy :", model.score(x_test, y_test))

cm = confusion_matrix(y_test, y_pred)
print(cm)

"""**Decision Forest**"""

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix

model = DecisionTreeClassifier()
model.fit(x_train, y_train)

y_pred = model.predict(x_test)

print("Training Accuracy :", model.score(x_train, y_train))
print("Testing Accuracy :", model.score(x_test, y_test))

cm = confusion_matrix(y_test, y_pred)
print(cm)

ModuleNotFoundError: No module named 'google.colab'

In [4]:
pip install --upgrade google-api-python-client


     ---------------------------------------- 12.2/12.2 MB 1.0 MB/s eta 0:00:00
     -------------------------------------- 137.0/137.0 kB 1.3 MB/s eta 0:00:00
     -------------------------------------- 186.9/186.9 kB 1.1 MB/s eta 0:00:00
     -------------------------------------- 96.9/96.9 kB 931.3 kB/s eta 0:00:00
     -------------------------------------- 228.7/228.7 kB 1.3 MB/s eta 0:00:00
     ------------------------------------ 413.4/413.4 kB 806.3 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install google-colab


     ---------------------------------------- 72.9/72.9 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 64.2/64.2 kB 1.7 MB/s eta 0:00:00
     -------------------------------------- 104.5/104.5 kB 2.9 MB/s eta 0:00:00
     -------------------------------------- 758.9/758.9 kB 4.0 MB/s eta 0:00:00
     ---------------------------------------- 8.0/8.0 MB 819.8 kB/s eta 0:00:00
     ---------------------------------------- 11.8/11.8 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 58.0/58.0 kB ? eta 0:00:00
     -------------------------------------- 484.2/484.2 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata

  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [1003 lines of output]
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-cpython-310
  creating build\lib.win-amd64-cpython-310\pandas
  copying pandas\conftest.py -> build\lib.win-amd64-cpython-310\pandas
  copying pandas\testing.py -> build\lib.win-amd64-cpython-310\pandas
  copying pandas\_version.py -> build\lib.win-amd64-cpython-310\pandas
  copying pandas\__init__.py -> build\lib.win-amd64-cpython-310\pandas
  creating build\lib.win-amd64-cpython-310\pandas\api
  copying pandas\api\__init__.py -> build\lib.win-amd64-cpython-310\pandas\api
  creating build\lib.win-amd64-cpython-310\pandas\arrays
  copying pandas\arrays\__init__.py -> build\lib.win-amd64-cpython-310\pandas\arrays
  creating build\lib.win-amd64-cpython-310\pandas\compat
  copying pandas\compat\chainmap.py -> build\lib.win-amd64-cpython-310\pandas\